<a href="https://colab.research.google.com/github/UmarFarooqui/Springboard/blob/master/Data_Wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from pprint import pprint
import json
from pathlib import Path
import pandas as pd
import requests
from google.colab import files
import subprocess

In [11]:
# Mounting the google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
# Uploaded the dataset to google drive. Checking.
!ls "/content/drive/My Drive/data"

2019-11-20-15.json.gz  codesearchnet_python.zip  repository_star_info.json


In [13]:
!unzip "/content/drive/My Drive/data/codesearchnet_python.zip"

Archive:  /content/drive/My Drive/data/codesearchnet_python.zip
replace python/final/jsonl/train/python_train_9.jsonl.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace python/final/jsonl/train/python_train_12.jsonl.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace python/final/jsonl/train/python_train_10.jsonl.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace python/final/jsonl/train/python_train_0.jsonl.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace python/final/jsonl/train/python_train_6.jsonl.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace python/final/jsonl/train/python_train_2.jsonl.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace python/final/jsonl/train/python_train_4.jsonl.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace python/final/jsonl/train/python_train_8.jsonl.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace python/final/jsonl/train/python_train_11.jsonl.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace python/final/jsonl/train/python_train_5.jso

In [0]:
# Exploring the full dataset
python_files = sorted(Path('python/').glob('**/*.gz'))

In [15]:
python_files

[PosixPath('python/final/jsonl/test/python_test_0.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_0.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_1.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_10.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_11.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_12.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_13.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_2.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_3.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_4.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_5.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_6.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_7.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_8.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_9.jsonl.gz'),
 PosixPath('python/final/jsonl/valid/p

In [16]:
print(f'Total number of files: {len(python_files)}')

Total number of files: 16


In [0]:
columns_list = ['repo', 'path', 'url', 'func_name', 'code', 'code_tokens', 'docstring', 'docstring_tokens']

In [0]:
def jsonl_list_to_dataframe(file_list, columns=columns_list):
  """Load a list of jsonl.gz files into a pandas dataframe."""
  return pd.concat([pd.read_json(f, orient='records', compression='gzip', lines=True)[columns] for f in file_list], sort=False) 

In [0]:
pydf = jsonl_list_to_dataframe(python_files)

In [20]:
pydf.head(3)

,repo,path,url,func_name,code,code_tokens,docstring,docstring_tokens
0,soimort/you-get,src/you_get/extractors/youtube.py,https://github.com/soimort/you-get/blob/b746ac...,YouTube.get_vid_from_url,"def get_vid_from_url(url):\n """"""Extract...","[def, get_vid_from_url, (, url, ), :, return, ...",Extracts video ID from URL.,"[Extracts, video, ID, from, URL, .]"
1,soimort/you-get,src/you_get/extractors/miomio.py,https://github.com/soimort/you-get/blob/b746ac...,sina_xml_to_url_list,"def sina_xml_to_url_list(xml_data):\n """"""st...","[def, sina_xml_to_url_list, (, xml_data, ), :,...",str->list\n Convert XML to URL List.\n F...,"[str, -, >, list, Convert, XML, to, URL, List,..."
2,soimort/you-get,src/you_get/extractors/fc2video.py,https://github.com/soimort/you-get/blob/b746ac...,makeMimi,"def makeMimi(upid):\n """"""From http://cdn37....","[def, makeMimi, (, upid, ), :, strSeed, =, ""gG...",From http://cdn37.atwikiimg.com/sitescript/pub...,"[From, http, :, //, cdn37, ., atwikiimg, ., co..."


In [21]:
# The columns which will be used for our purpose would be func_name, code, code_tokens, docstring, docstring_tokens
pydf.groupby(['repo'])['code'].count()

repo
0101/pipetools                   12
02strich/django-auth-kerberos     1
05bit/peewee-async               42
0compute/xtraceback               2
0k/kids.cache                     4
                                 ..
zyga/python-glibc                32
zyga/python-phablet               6
zzyztyy/pyIGRF                    6
zzzsochi/includer                 2
zzzsochi/resolver_deco            1
Name: code, Length: 13590, dtype: int64

In [22]:
# As a starting point we will use the repos with a high star count
# Example of how to do it
url = 'http://api.github.com/repos/'
repo_name = '0101/pipetools'
r = requests.get(url + repo_name)
r_json = r.json()
r_json['stargazers_count']

81

In [23]:
# Now we will add this as the last column in our dataframe
# First check if we have no null values in the repo column
pydf.repo.isnull().any()

False

In [24]:
pydf[pydf['repo'] == '']

,repo,path,url,func_name,code,code_tokens,docstring,docstring_tokens


In [25]:
# So looks like the repo column is having no null values or empty string
# Taking a look at the dataframe
pydf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 457461 entries, 0 to 23106
Data columns (total 8 columns):
repo                457461 non-null object
path                457461 non-null object
url                 457461 non-null object
func_name           457461 non-null object
code                457461 non-null object
code_tokens         457461 non-null object
docstring           457461 non-null object
docstring_tokens    457461 non-null object
dtypes: object(8)
memory usage: 31.4+ MB


In [26]:
# Looks like we have no null values
# Length of the dataframe
len(pydf)

457461

In [27]:
# So we have 0.45M samples, we will increase our dataset if we need to by exploring more repos.
unique_repo_name_series = pydf.repo.drop_duplicates()
unique_repo_name_series

0                       soimort/you-get
52                       apache/airflow
849                      pytorch/vision
900                      asciimoo/searx
910              tensorflow/probability
                      ...              
23018           praekelt/python-ambient
23024                   zenreach/py-era
23090    TakesxiSximada/custom_settings
23095              openpermissions/bass
23099                 xnuinside/clifier
Name: repo, Length: 13590, dtype: object

In [28]:
# Uploaded the downloaded file to the gdrive, checking if the system recognizes it
!ls "/content/drive/My Drive/data"

2019-11-20-15.json.gz  codesearchnet_python.zip  repository_star_info.json


In [29]:
# Reading the json from the file
with open('/content/drive/My Drive/data/repository_star_info.json') as json_file:
  repository_metadata_info = json.load(json_file)
repository_metadata_info

{'3DLIRIOUS/MeshLabXML': 117,
 'Azure/azure-sdk-for-python': 909,
 'CITGuru/PyInquirer': 561,
 'Danielhiversen/PyXiaomiGateway': 102,
 'Microsoft/botbuilder-python': 265,
 'Nekmo/amazon-dash': 567,
 'PyCQA/pylint': 2417,
 'Qiskit/qiskit-terra': 2634,
 'SectorLabs/django-postgres-extra': 254,
 'Tivix/django-rest-auth': 1848,
 'alpacahq/alpaca-trade-api-python': 224,
 'ansible/ansible-lint': 1859,
 'apache/airflow': 14535,
 'asciimoo/searx': 5598,
 'bloomreach/s4cmd': 929,
 'bluedisk/hangul-toolkit': 154,
 'celiao/tmdbsimple': 250,
 'chaoss/grimoirelab-perceval': 140,
 'dagster-io/dagster': 983,
 'deepmipt/DeepPavlov': 3668,
 'ekzhu/datasketch': 1087,
 'geopython/OWSLib': 187,
 'google/brotli': 7916,
 'google/gin-config': 581,
 'graphql-python/graphene-sqlalchemy': 603,
 'h2oai/h2o-3': 4434,
 'howie6879/ruia': 1028,
 'instagrambot/instabot': 2705,
 'intel-analytics/BigDL': 3182,
 'kennethreitz/twitter-scraper': 1886,
 'lensacom/sparkit-learn': 989,
 'lepture/flask-oauthlib': 1384,
 'libr

In [30]:
# Append a column in the pydf dataframe called star that contains the star information we have discovered so far
pydf.head(3)

,repo,path,url,func_name,code,code_tokens,docstring,docstring_tokens
0,soimort/you-get,src/you_get/extractors/youtube.py,https://github.com/soimort/you-get/blob/b746ac...,YouTube.get_vid_from_url,"def get_vid_from_url(url):\n """"""Extract...","[def, get_vid_from_url, (, url, ), :, return, ...",Extracts video ID from URL.,"[Extracts, video, ID, from, URL, .]"
1,soimort/you-get,src/you_get/extractors/miomio.py,https://github.com/soimort/you-get/blob/b746ac...,sina_xml_to_url_list,"def sina_xml_to_url_list(xml_data):\n """"""st...","[def, sina_xml_to_url_list, (, xml_data, ), :,...",str->list\n Convert XML to URL List.\n F...,"[str, -, >, list, Convert, XML, to, URL, List,..."
2,soimort/you-get,src/you_get/extractors/fc2video.py,https://github.com/soimort/you-get/blob/b746ac...,makeMimi,"def makeMimi(upid):\n """"""From http://cdn37....","[def, makeMimi, (, upid, ), :, strSeed, =, ""gG...",From http://cdn37.atwikiimg.com/sitescript/pub...,"[From, http, :, //, cdn37, ., atwikiimg, ., co..."


In [31]:
pydf_with_star = pydf
pydf_with_star['star'] = ""
for index, row in pydf_with_star.iterrows():
  if row['repo'] in repository_metadata_info:
    row['star'] = repository_metadata_info[row['repo']]
pydf_with_star

,repo,path,url,func_name,code,code_tokens,docstring,docstring_tokens,star
0,soimort/you-get,src/you_get/extractors/youtube.py,https://github.com/soimort/you-get/blob/b746ac...,YouTube.get_vid_from_url,"def get_vid_from_url(url):\n """"""Extract...","[def, get_vid_from_url, (, url, ), :, return, ...",Extracts video ID from URL.,"[Extracts, video, ID, from, URL, .]",27926
1,soimort/you-get,src/you_get/extractors/miomio.py,https://github.com/soimort/you-get/blob/b746ac...,sina_xml_to_url_list,"def sina_xml_to_url_list(xml_data):\n """"""st...","[def, sina_xml_to_url_list, (, xml_data, ), :,...",str->list\n Convert XML to URL List.\n F...,"[str, -, >, list, Convert, XML, to, URL, List,...",27926
2,soimort/you-get,src/you_get/extractors/fc2video.py,https://github.com/soimort/you-get/blob/b746ac...,makeMimi,"def makeMimi(upid):\n """"""From http://cdn37....","[def, makeMimi, (, upid, ), :, strSeed, =, ""gG...",From http://cdn37.atwikiimg.com/sitescript/pub...,"[From, http, :, //, cdn37, ., atwikiimg, ., co...",27926
3,soimort/you-get,src/you_get/extractors/fc2video.py,https://github.com/soimort/you-get/blob/b746ac...,fc2video_download,"def fc2video_download(url, output_dir = '.', m...","[def, fc2video_download, (, url, ,, output_dir...",wrapper,[wrapper],27926
4,soimort/you-get,src/you_get/extractors/dailymotion.py,https://github.com/soimort/you-get/blob/b746ac...,dailymotion_download,"def dailymotion_download(url, output_dir='.', ...","[def, dailymotion_download, (, url, ,, output_...",Downloads Dailymotion videos by URL.,"[Downloads, Dailymotion, videos, by, URL, .]",27926
...,...,...,...,...,...,...,...,...,...
23102,xnuinside/clifier,clifier/clifier.py,https://github.com/xnuinside/clifier/blob/3d70...,Clifier.show_version,"def show_version(self):\n """""" custom co...","[def, show_version, (, self, ), :, class, Show...",custom command line action to show version,"[custom, command, line, action, to, show, vers...",
23103,xnuinside/clifier,clifier/clifier.py,https://github.com/xnuinside/clifier/blob/3d70...,Clifier.check_path_action,"def check_path_action(self):\n """""" cust...","[def, check_path_action, (, self, ), :, class,...",custom command line action to check file exist,"[custom, command, line, action, to, check, fil...",
23104,tklovett/PyShirtsIO,interactive_console.py,https://github.com/tklovett/PyShirtsIO/blob/ff...,new_user,def new_user(yaml_path):\n '''\n Return ...,"[def, new_user, (, yaml_path, ), :, print, 'Re...",Return the consumer and oauth tokens with thre...,"[Return, the, consumer, and, oauth, tokens, wi...",
23105,ibelie/typy,typy/google/protobuf/internal/python_message.py,https://github.com/ibelie/typy/blob/3616845fb9...,_AddPropertiesForExtensions,"def _AddPropertiesForExtensions(descriptor, cl...","[def, _AddPropertiesForExtensions, (, descript...",Adds properties for all fields in this protoco...,"[Adds, properties, for, all, fields, in, this,...",


In [32]:
# Lets sort on the stars to see what are the high ranked repos we have so far
pydf_valid_star_df = pydf_with_star[pydf_with_star['star'] != '']
pydf_valid_star_df

,repo,path,url,func_name,code,code_tokens,docstring,docstring_tokens,star
0,soimort/you-get,src/you_get/extractors/youtube.py,https://github.com/soimort/you-get/blob/b746ac...,YouTube.get_vid_from_url,"def get_vid_from_url(url):\n """"""Extract...","[def, get_vid_from_url, (, url, ), :, return, ...",Extracts video ID from URL.,"[Extracts, video, ID, from, URL, .]",27926
1,soimort/you-get,src/you_get/extractors/miomio.py,https://github.com/soimort/you-get/blob/b746ac...,sina_xml_to_url_list,"def sina_xml_to_url_list(xml_data):\n """"""st...","[def, sina_xml_to_url_list, (, xml_data, ), :,...",str->list\n Convert XML to URL List.\n F...,"[str, -, >, list, Convert, XML, to, URL, List,...",27926
2,soimort/you-get,src/you_get/extractors/fc2video.py,https://github.com/soimort/you-get/blob/b746ac...,makeMimi,"def makeMimi(upid):\n """"""From http://cdn37....","[def, makeMimi, (, upid, ), :, strSeed, =, ""gG...",From http://cdn37.atwikiimg.com/sitescript/pub...,"[From, http, :, //, cdn37, ., atwikiimg, ., co...",27926
3,soimort/you-get,src/you_get/extractors/fc2video.py,https://github.com/soimort/you-get/blob/b746ac...,fc2video_download,"def fc2video_download(url, output_dir = '.', m...","[def, fc2video_download, (, url, ,, output_dir...",wrapper,[wrapper],27926
4,soimort/you-get,src/you_get/extractors/dailymotion.py,https://github.com/soimort/you-get/blob/b746ac...,dailymotion_download,"def dailymotion_download(url, output_dir='.', ...","[def, dailymotion_download, (, url, ,, output_...",Downloads Dailymotion videos by URL.,"[Downloads, Dailymotion, videos, by, URL, .]",27926
...,...,...,...,...,...,...,...,...,...
6547,Nekmo/amazon-dash,amazon_dash/listener.py,https://github.com/Nekmo/amazon-dash/blob/0e2b...,Device.send_confirmation,"def send_confirmation(self, message, success=T...","[def, send_confirmation, (, self, ,, message, ...",Send success or error message to configured co...,"[Send, success, or, error, message, to, config...",567
6548,Nekmo/amazon-dash,amazon_dash/listener.py,https://github.com/Nekmo/amazon-dash/blob/0e2b...,Listener.on_push,"def on_push(self, device):\n """"""Press b...","[def, on_push, (, self, ,, device, ), :, src, ...",Press button. Check DEFAULT_DELAY.\n\n ...,"[Press, button, ., Check, DEFAULT_DELAY, .]",567
6549,Nekmo/amazon-dash,amazon_dash/listener.py,https://github.com/Nekmo/amazon-dash/blob/0e2b...,Listener.execute,"def execute(self, device):\n """"""Execute...","[def, execute, (, self, ,, device, ), :, src, ...",Execute a device. Used if the time between exe...,"[Execute, a, device, ., Used, if, the, time, b...",567
6550,Nekmo/amazon-dash,amazon_dash/listener.py,https://github.com/Nekmo/amazon-dash/blob/0e2b...,Listener.run,"def run(self, root_allowed=False):\n """"...","[def, run, (, self, ,, root_allowed, =, False,...",Start daemon mode\n\n :param bool root_...,"[Start, daemon, mode]",567


In [33]:
pydf_valid_star_df['star'].max()

27926

In [34]:
pydf_valid_star_df['star'].min()

75

In [35]:
# For testing, choosing the top rated repository
pydf_highest_star_df = pydf_valid_star_df[pydf_valid_star_df['star'] == pydf_valid_star_df['star'].max()]
pydf_highest_star_df

,repo,path,url,func_name,code,code_tokens,docstring,docstring_tokens,star
0,soimort/you-get,src/you_get/extractors/youtube.py,https://github.com/soimort/you-get/blob/b746ac...,YouTube.get_vid_from_url,"def get_vid_from_url(url):\n """"""Extract...","[def, get_vid_from_url, (, url, ), :, return, ...",Extracts video ID from URL.,"[Extracts, video, ID, from, URL, .]",27926
1,soimort/you-get,src/you_get/extractors/miomio.py,https://github.com/soimort/you-get/blob/b746ac...,sina_xml_to_url_list,"def sina_xml_to_url_list(xml_data):\n """"""st...","[def, sina_xml_to_url_list, (, xml_data, ), :,...",str->list\n Convert XML to URL List.\n F...,"[str, -, >, list, Convert, XML, to, URL, List,...",27926
2,soimort/you-get,src/you_get/extractors/fc2video.py,https://github.com/soimort/you-get/blob/b746ac...,makeMimi,"def makeMimi(upid):\n """"""From http://cdn37....","[def, makeMimi, (, upid, ), :, strSeed, =, ""gG...",From http://cdn37.atwikiimg.com/sitescript/pub...,"[From, http, :, //, cdn37, ., atwikiimg, ., co...",27926
3,soimort/you-get,src/you_get/extractors/fc2video.py,https://github.com/soimort/you-get/blob/b746ac...,fc2video_download,"def fc2video_download(url, output_dir = '.', m...","[def, fc2video_download, (, url, ,, output_dir...",wrapper,[wrapper],27926
4,soimort/you-get,src/you_get/extractors/dailymotion.py,https://github.com/soimort/you-get/blob/b746ac...,dailymotion_download,"def dailymotion_download(url, output_dir='.', ...","[def, dailymotion_download, (, url, ,, output_...",Downloads Dailymotion videos by URL.,"[Downloads, Dailymotion, videos, by, URL, .]",27926
5,soimort/you-get,src/you_get/extractors/ucas.py,https://github.com/soimort/you-get/blob/b746ac...,dictify,"def dictify(r,root=True):\n """"""http://stack...","[def, dictify, (, r, ,, root, =, True, ), :, i...",http://stackoverflow.com/a/30923963/2946714,"[http, :, //, stackoverflow, ., com, /, a, /, ...",27926
6,soimort/you-get,src/you_get/extractors/ucas.py,https://github.com/soimort/you-get/blob/b746ac...,ucas_download_single,"def ucas_download_single(url, output_dir = '.'...","[def, ucas_download_single, (, url, ,, output_...",video page,"[video, page]",27926
7,soimort/you-get,src/you_get/extractors/ucas.py,https://github.com/soimort/you-get/blob/b746ac...,ucas_download_playlist,"def ucas_download_playlist(url, output_dir = '...","[def, ucas_download_playlist, (, url, ,, outpu...",course page,"[course, page]",27926
8,soimort/you-get,src/you_get/extractors/sina.py,https://github.com/soimort/you-get/blob/b746ac...,sina_download_by_vid,"def sina_download_by_vid(vid, title=None, outp...","[def, sina_download_by_vid, (, vid, ,, title, ...",Downloads a Sina video by its unique vid.\n ...,"[Downloads, a, Sina, video, by, its, unique, v...",27926
9,soimort/you-get,src/you_get/extractors/sina.py,https://github.com/soimort/you-get/blob/b746ac...,sina_download_by_vkey,"def sina_download_by_vkey(vkey, title=None, ou...","[def, sina_download_by_vkey, (, vkey, ,, title...",Downloads a Sina video by its unique vkey.\n ...,"[Downloads, a, Sina, video, by, its, unique, v...",27926


In [36]:
pydf_highest_star_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52 entries, 0 to 51
Data columns (total 9 columns):
repo                52 non-null object
path                52 non-null object
url                 52 non-null object
func_name           52 non-null object
code                52 non-null object
code_tokens         52 non-null object
docstring           52 non-null object
docstring_tokens    52 non-null object
star                52 non-null object
dtypes: object(9)
memory usage: 4.1+ KB


In [37]:
# Checking how many unique repositories are having the highest star value we got from last step
pydf_highest_star_df.repo.unique()

array(['soimort/you-get'], dtype=object)

In [38]:
# Now lets get the AST path representation of this dataframe using the AST miner tool
# The command is: java -jar cli.jar code2vec --lang py --project path/to/project --output path/to/results --maxH H --maxW W --maxContexts C --maxTokens T --maxPaths P
# Uploaded the astminer cli jar file to the gdrive, checking if the system recognizes it
!ls "/content/drive/My Drive/astminer/"

cli.jar  code  code_astminer_result  sample


In [39]:
# Checking for the presence of java JDK
!which java

/usr/bin/java


In [40]:
# Checking the version of java 
!java --version

openjdk 11.0.4 2019-07-16
OpenJDK Runtime Environment (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3)
OpenJDK 64-Bit Server VM (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3, mixed mode, sharing)


In [41]:
# Running the jar file without any argument to see if it works
!java -jar "/content/drive/My Drive/astminer/cli.jar" code2vec

Usage: code2vecextractor [OPTIONS]

Error: Missing option "--methodContent".


In [0]:
# Remove the previous results
!rm -rf "/content/drive/My Drive/astminer/sample/HelloWorldProjectResults/*"

In [43]:
# See how the code looks like
method_content = pydf_highest_star_df.loc[0, 'code']
method_content

'def get_vid_from_url(url):\n        """Extracts video ID from URL.\n        """\n        return match1(url, r\'youtu\\.be/([^?/]+)\') or \\\n          match1(url, r\'youtube\\.com/embed/([^/?]+)\') or \\\n          match1(url, r\'youtube\\.com/v/([^/?]+)\') or \\\n          match1(url, r\'youtube\\.com/watch/([^/?]+)\') or \\\n          parse_query_param(url, \'v\') or \\\n          parse_query_param(parse_query_param(url, \'u\'), \'v\')'

In [0]:
def get_path_context_for_method(method_content):
  comm = ['java', '-jar', '/content/drive/My Drive/astminer/cli.jar', 'code2vec', '--methodContent', method_content]
  try:
    output_msg = subprocess.check_output(comm, stderr=subprocess.STDOUT)
    return output_msg.decode("utf-8").splitlines()
  except subprocess.CalledProcessError as e:
    return None

In [45]:
if get_path_context_for_method(method_content):
  result = get_path_context_for_method(method_content)
  print(result[0])

[get|vid|from|url 1,1,2 1,2,3 2,3,3 4,4,5 2,5,6 2,6,7 6,7,7 1,8,2 1,9,6 1,10,7 1,11,3 2,12,3 6,13,3 7,14,3 8,15,1 8,16,2 8,17,6 8,18,7 8,19,3 2,5,6 2,6,9 6,7,9 1,8,2 1,9,6 1,10,9 1,11,3 2,12,3 6,13,3 9,14,3 8,15,1 8,16,2 8,17,6 8,18,9 8,19,3 2,5,6 2,6,10 6,7,10 1,8,2 1,9,6 1,10,10 1,11,3 2,12,3 6,13,3 10,14,3 8,15,1 8,16,2 8,17,6 8,18,10 8,19,3 2,5,6 2,6,11 6,7,11 1,8,2 1,9,6 1,10,11 1,11,3 2,12,3 6,13,3 11,14,3 8,15,1 8,16,2 8,17,6 8,18,11 8,19,3 2,5,6 2,6,12 6,7,12 1,8,2 1,9,6 1,10,12 1,11,3 2,12,3 6,13,3 12,14,3 13,15,1 13,16,2 13,17,6 13,18,12 13,19,3 2,5,6 2,6,14 6,7,14 1,8,2 1,9,6 1,10,14 1,11,3 2,12,3 6,13,3 14,14,3 13,20,1 13,21,2 13,22,6 13,23,14 13,24,3 1,25,6 2,26,6 2,27,12 6,28,6 6,29,12 14,30,6 14,31,12 3,32,6 3,33,12 6,7,12 1,34,13 1,35,1 1,36,2 1,37,6 1,38,14 6,39,3 14,40,3 3,41,3 6,13,3 12,14,3 13,15,1 13,42,13 13,43,1 1,44,15 2,45,15 2,46,8 6,47,15 6,48,8 6,49,1 7,50,15 7,51,8 7,52,1 7,53,2 3,54,15 3,55,8 3,56,1 3,57,2 3,58,6 15,59,8 15,60,1 15,61,2 15,62,6 15,63,9 1,4

In [46]:
# For testing, lets create a dataframe having the path contexts in a column
pydf_highest_star_with_path_contexts_df = pydf_highest_star_df
pydf_highest_star_with_path_contexts_df.loc[:, ('path_contexts')] = ''
pydf_highest_star_with_path_contexts_df.loc[:, ('tokens')] = ''
pydf_highest_star_with_path_contexts_df.loc[:, ('node_types')] = ''
pydf_highest_star_with_path_contexts_df.loc[:, ('paths')] = ''
for index, row in pydf_highest_star_with_path_contexts_df.iterrows():
  if not row['path_contexts']:
    if get_path_context_for_method(row['code']):
      row['path_contexts'] = get_path_context_for_method(row['code'])[0]
      row['tokens'] = get_path_context_for_method(row['code'])[1]
      row['node_types'] = get_path_context_for_method(row['code'])[2]
      row['paths'] = get_path_context_for_method(row['code'])[3]

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [47]:
pydf_highest_star_with_path_contexts_df

,repo,path,url,func_name,code,code_tokens,docstring,docstring_tokens,star,path_contexts,tokens,node_types,paths
0,soimort/you-get,src/you_get/extractors/youtube.py,https://github.com/soimort/you-get/blob/b746ac...,YouTube.get_vid_from_url,"def get_vid_from_url(url):\n """"""Extract...","[def, get_vid_from_url, (, url, ), :, return, ...",Extracts video ID from URL.,"[Extracts, video, ID, from, URL, .]",27926,"[get|vid|from|url 1,1,2 1,2,3 2,3,3 4,4,5 2,5,...","[id,token, 15,or, 17,________, 11,ryoutubecomw...","[id,node_type, 34,small_stmt|flow_stmt|return_...","[id,path, 106,27 2 45 46 38 39, 47,16 12 17 22..."
1,soimort/you-get,src/you_get/extractors/miomio.py,https://github.com/soimort/you-get/blob/b746ac...,sina_xml_to_url_list,"def sina_xml_to_url_list(xml_data):\n """"""st...","[def, sina_xml_to_url_list, (, xml_data, ), :,...",str->list\n Convert XML to URL List.\n F...,"[str, -, >, list, Convert, XML, to, URL, List,...",27926,"[sina|xml|to|url|list 1,1,2 1,2,3 2,3,3 4,4,5 ...","[id,token, 11,dom, 19,childnodes, 21,________,...","[id,node_type, 23,trailer DOWN, 28,testlist_st...","[id,path, 128,31 22 44 8 9 10, 272,61 72 73 48..."
2,soimort/you-get,src/you_get/extractors/fc2video.py,https://github.com/soimort/you-get/blob/b746ac...,makeMimi,"def makeMimi(upid):\n """"""From http://cdn37....","[def, makeMimi, (, upid, ), :, strSeed, =, ""gG...",From http://cdn37.atwikiimg.com/sitescript/pub...,"[From, http, :, //, cdn37, ., atwikiimg, ., co...",27926,"[make|mimi 1,1,2 1,2,3 2,3,3 4,4,5 6,5,7 6,6,8...","[id,token, 2,upid, 12,encode, 4,fromhttpcdnatw...","[id,node_type, 14,ASSIGN DOWN, 49,small_stmt|e...","[id,path, 112,45 8 46 47 9 13 20 23, 145,6 2 5..."
3,soimort/you-get,src/you_get/extractors/fc2video.py,https://github.com/soimort/you-get/blob/b746ac...,fc2video_download,"def fc2video_download(url, output_dir = '.', m...","[def, fc2video_download, (, url, ,, output_dir...",wrapper,[wrapper],27926,"[fc|video|download 1,1,2 1,2,3 1,3,4 1,4,5 1,1...","[id,token, 10,**, 29,upid, 3,outputdir, 24,___...","[id,node_type, 43,and_test|not_test|comparison...","[id,path, 72,29 25 32 38 22 23 24, 118,64 22 6..."
4,soimort/you-get,src/you_get/extractors/dailymotion.py,https://github.com/soimort/you-get/blob/b746ac...,dailymotion_download,"def dailymotion_download(url, output_dir='.', ...","[def, dailymotion_download, (, url, ,, output_...",Downloads Dailymotion videos by URL.,"[Downloads, Dailymotion, videos, by, URL, .]",27926,"[dailymotion|download 1,1,2 1,2,3 1,3,4 1,4,5 ...","[id,token, 35,auto, 50,________, 16,rebuilturl...","[id,node_type, 70,small_stmt|flow_stmt|break_s...","[id,path, 318,104 99 100 60 57 4, 154,28 25 35..."
5,soimort/you-get,src/you_get/extractors/ucas.py,https://github.com/soimort/you-get/blob/b746ac...,dictify,"def dictify(r,root=True):\n """"""http://stack...","[def, dictify, (, r, ,, root, =, True, ), :, i...",http://stackoverflow.com/a/30923963/2946714,"[http, :, //, stackoverflow, ., com, /, a, /, ...",27926,"[dictify 1,1,2 1,2,3 1,3,4 1,4,5 2,5,3 2,6,4 2...","[id,token, 13,dictify, 18,________, 35,def, 33...","[id,node_type, 99,FOR UP, 59,stmt|compound_stm...","[id,path, 339,39 100 101 52 53, 393,45 46 16 5..."
6,soimort/you-get,src/you_get/extractors/ucas.py,https://github.com/soimort/you-get/blob/b746ac...,ucas_download_single,"def ucas_download_single(url, output_dir = '.'...","[def, ucas_download_single, (, url, ,, output_...",video page,"[video, page]",27926,"[ucas|download|single 1,1,2 1,2,3 1,3,4 1,4,5 ...","[id,token, 18,rresourceidafafafafaf, 49,______...","[id,node_type, 25,arglist|argument|test|or_tes...","[id,path, 94,35 23 28 29 24 50, 124,8 59 20 21..."
7,soimort/you-get,src/you_get/extractors/ucas.py,https://github.com/soimort/you-get/blob/b746ac...,ucas_download_playlist,"def ucas_download_playlist(url, output_dir = '...","[def, ucas_download_playlist, (, url, ,, outpu...",course page,"[course, page]",27926,"[ucas|download|playlist 1,1,2 1,2,3 1,3,4 1,4,...","[id,token, 9,**, 3,outputd

In [48]:
# Confirm that we have the first entry correct
pydf_highest_star_df.loc[0, 'path_contexts']

'[get|vid|from|url 1,1,2 1,2,3 2,3,3 4,4,5 2,5,6 2,6,7 6,7,7 1,8,2 1,9,6 1,10,7 1,11,3 2,12,3 6,13,3 7,14,3 8,15,1 8,16,2 8,17,6 8,18,7 8,19,3 2,5,6 2,6,9 6,7,9 1,8,2 1,9,6 1,10,9 1,11,3 2,12,3 6,13,3 9,14,3 8,15,1 8,16,2 8,17,6 8,18,9 8,19,3 2,5,6 2,6,10 6,7,10 1,8,2 1,9,6 1,10,10 1,11,3 2,12,3 6,13,3 10,14,3 8,15,1 8,16,2 8,17,6 8,18,10 8,19,3 2,5,6 2,6,11 6,7,11 1,8,2 1,9,6 1,10,11 1,11,3 2,12,3 6,13,3 11,14,3 8,15,1 8,16,2 8,17,6 8,18,11 8,19,3 2,5,6 2,6,12 6,7,12 1,8,2 1,9,6 1,10,12 1,11,3 2,12,3 6,13,3 12,14,3 13,15,1 13,16,2 13,17,6 13,18,12 13,19,3 2,5,6 2,6,14 6,7,14 1,8,2 1,9,6 1,10,14 1,11,3 2,12,3 6,13,3 14,14,3 13,20,1 13,21,2 13,22,6 13,23,14 13,24,3 1,25,6 2,26,6 2,27,12 6,28,6 6,29,12 14,30,6 14,31,12 3,32,6 3,33,12 6,7,12 1,34,13 1,35,1 1,36,2 1,37,6 1,38,14 6,39,3 14,40,3 3,41,3 6,13,3 12,14,3 13,15,1 13,42,13 13,43,1 1,44,15 2,45,15 2,46,8 6,47,15 6,48,8 6,49,1 7,50,15 7,51,8 7,52,1 7,53,2 3,54,15 3,55,8 3,56,1 3,57,2 3,58,6 15,59,8 15,60,1 15,61,2 15,62,6 15,63,9 1,

In [49]:
pydf_highest_star_df.loc[0, 'tokens']

'[id,token, 15,or, 17,________, 11,ryoutubecomwatch, 18,def, 1,(, 8,match, 3,), 6,EMPTY_TOKEN, 9,ryoutubecomembed, 10,ryoutubecomv, 2,url, 13,parsequeryparam, 4,extractsvideoidfromurl, 14,u, 12,v, 7,ryoutube, 19,METHOD_NAME, 20,:, 16,return, 5,_]'

In [50]:
pydf_highest_star_df.loc[0, 'node_types']

'[id,node_type, 34,small_stmt|flow_stmt|return_stmt UP, 35,small_stmt|flow_stmt|return_stmt DOWN, 47,NAME DOWN, 31,OR DOWN, 32,OR UP, 49,NAME UP, 27,CLOSE_PAREN UP, 45,stmt|compound_stmt|funcdef UP, 46,stmt|compound_stmt|funcdef DOWN, 5,CLOSE_PAREN DOWN, 8,stmt|simple_stmt UP, 9,stmt|simple_stmt DOWN, 21,atom|NAME UP, 28,atom|NAME DOWN, 50,COLON UP, 10,NEWLINE DOWN, 19,argument|test|or_test|and_test|not_test|comparison|expr|xor_expr|and_expr|shift_expr|arith_expr|term|factor|power|atom_expr|atom|NAME DOWN, 36,NEWLINE UP, 11,argument|test|or_test|and_test|not_test|comparison|expr|xor_expr|and_expr|shift_expr|arith_expr|term|factor|power|atom_expr|atom|NAME UP, 7,small_stmt|expr_stmt|testlist_star_expr|test|or_test|and_test|not_test|comparison|expr|xor_expr|and_expr|shift_expr|arith_expr|term|factor|power|atom_expr|atom|STRING UP, 40,small_stmt|expr_stmt|testlist_star_expr|test|or_test|and_test|not_test|comparison|expr|xor_expr|and_expr|shift_expr|arith_expr|term|factor|power|atom_expr|a

In [51]:
pydf_highest_star_df.loc[0, 'paths']

'[id,path, 106,27 2 45 46 38 39, 47,16 12 17 22 29 30 31, 7,16 12 13 15, 62,32 29 30 23 18 13 14, 63,32 29 30 23 18 13 15, 64,20 12 17 22 29 30 23 18 13 26 28, 19,21 22 23 18 5, 58,27 17 22 29 30 23 18 13 14, 40,20 12 17 25 12 17 18 5, 39,16 12 17 25 12 17 18 5, 100,6 2 45 46 48, 15,21 22 23 18 24, 112,50 45 46 38 9 10, 36,1 17 18 13 26 18 13 19, 105,27 2 45 46 38 10, 96,49 45 46 38 10, 98,1 2 45 46 38 10, 38,1 17 18 13 26 18 13 15, 37,1 17 18 13 26 18 13 14, 61,32 29 30 23 18 13 19, 43,21 22 23 18 13 26 18 24, 111,50 45 46 38 9 40, 53,20 12 17 22 29 30 23 18 13 19, 67,33 34 35 30 23 28, 97,1 2 45 46 48, 102,6 2 45 46 38 39, 45,11 12 17 22 29 30 31, 68,33 34 35 30 23 18 24, 5,11 12 13 14, 4,7 8 9 10, 28,16 12 17 25 12 13 14, 29,16 12 17 25 12 13 15, 52,20 12 17 22 29 30 23 18 24, 101,6 2 45 46 38 10, 6,11 12 13 15, 60,32 29 30 23 18 24, 92,49 45 46 3 24, 108,27 2 45 46 38 9 10, 109,50 45 46 38 10, 79,42 37 38 9 10, 82,7 8 37 38 9 35 41, 49,16 12 17 22 29 30 23 18 24, 72,27 17 22 29 34 

In [52]:
# Now lets explore the metadata from GH Archive. May be we can get the repository's information in that dataset and then we can avoid the need to do a lot of rest api calls to github endpoint.
# Activity for 11/21/2015 @ 3PM UTC	wget https://data.gharchive.org/2019-11-20-15.json.gz
!ls "/content/drive/My Drive/data"

2019-11-20-15.json.gz  codesearchnet_python.zip  repository_star_info.json


In [53]:
gharchive_df = pd.read_json('/content/drive/My Drive/data/2019-11-20-15.json.gz', compression='gzip', lines=True)
gharchive_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117813 entries, 0 to 117812
Data columns (total 8 columns):
id            117813 non-null int64
type          117813 non-null object
actor         117813 non-null object
repo          117813 non-null object
payload       117813 non-null object
public        117813 non-null bool
created_at    117813 non-null datetime64[ns, UTC]
org           41279 non-null object
dtypes: bool(1), datetime64[ns, UTC](1), int64(1), object(5)
memory usage: 6.4+ MB


In [54]:
gharchive_df.head()

,id,type,actor,repo,payload,public,created_at,org
0,10907531140,PushEvent,"{'id': 39841583, 'login': 'mashafrancis', 'dis...","{'id': 222937522, 'name': 'mashafrancis/butter...","{'push_id': 4298635215, 'size': 1, 'distinct_s...",True,2019-11-20 15:00:00+00:00,NaN
1,10907531144,PushEvent,"{'id': 52970249, 'login': 'JayIssuree', 'displ...","{'id': 222924853, 'name': 'JayIssuree/GildedRo...","{'push_id': 4298635208, 'size': 1, 'distinct_s...",True,2019-11-20 15:00:00+00:00,NaN
2,10907531145,PushEvent,"{'id': 57961738, 'login': 'goneHaywire', 'disp...","{'id': 222764554, 'name': 'goneHaywire/laravue...","{'push_id': 4298635211, 'size': 1, 'distinct_s...",True,2019-11-20 15:00:00+00:00,NaN
3,10907531146,PushEvent,"{'id': 12509673, 'login': 'peterzuger', 'displ...","{'id': 115456917, 'name': 'peterzuger/.emacs.d...","{'push_id': 4298635214, 'size': 1, 'distinct_s...",True,2019-11-20 15:00:00+00:00,NaN
4,10907531150,PushEvent,"{'id': 15221018, 'login': 'Ne00n', 'display_lo...","{'id': 173624835, 'name': 'Ne00n/Hetzner', 'ur...","{'push_id': 4298635216, 'size': 1, 'distinct_s...",True,2019-11-20 15:00:00+00:00,NaN
